<a href="https://colab.research.google.com/github/piruvari/practica_python/blob/master/parseter_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
developer=""" "" """
endereco_source="drive/MyDrive/carrefour/scripts/template/"
endereco_destino="drive/MyDrive/carrefour/scripts/destino/"
arquivos_a_processar=[{'arquivo':'2_clean_tabela.txt'},{'arquivo':'1_raw_tabela.txt'},{'arquivo':'4_trusted_hist_tabela.txt'},{'arquivo':'3_trusted_tabela.txt'},{'arquivo':'6_drop.txt'},{'arquivo':'7_insert.txt'},{'arquivo':'5_backup.txt'} ]

#tsys_cpaymentsliplinks

In [ ]:
tabela='tsys_cpaymentsliplinks'#'tsys_cpaymentsliplinks' #tsys_cstatementlinks, tsys_cstatements.
unique_key='paymentslipserno,trxnserno'
unique_key_lista=""" "paymentslipserno","trxnserno" """
scheduler=tabela
data_number='202403100000'
path='gs://csf-core-data-prd-transient/tsys_cpaymentsliplinks/R06_CL7496_BR_DWH_IssFeed_cpaymentsliplinks'

#tsys_cstatementlinks

In [ ]:
tabela='tsys_cstatementlinks'
unique_key='statement_serno,trxnserno'
unique_key_lista=""" "statement_serno","trxnserno" """
scheduler=tabela
data_number='202403080000'
path='gs://csf-core-data-prd-transient/tsys_cstatementlinks/R06_CL7496_BR_DWH_IssFeed_cstatementlinks'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
##tratamento para a estrutura da coluna
estrutura_column=""" {name: '#nomecoluna#', type: '#tipocoluna#'}, """
description_column=""" #nomecoluna#: '#descripcaocoluna#', """
case_column="""CASE WHEN type = 'D' AND #nomecoluna#  IS NULL THEN LEAD( #nomecoluna# ) OVER(PARTITION BY #unique_key# ORDER BY ingestion_ref_date DESC) ELSE  #nomecoluna#  end as  #nomecoluna# ,"""
columns=''
columns_description=''
table_description=''
columna_raw=''
count=0
columna_case=''
with open(endereco_source+'doc_tsys_'+tabela+'.txt', "r") as arquivo:
    linha = arquivo.readline()
    while linha:
        case_column_ax=case_column
        estrutura_column_ax=estrutura_column
        description_column_ax=description_column
        if count==0 :
            table_description=linha.replace('Layout do Registro: ','').replace('\n','')
            print(linha)
        else:
            print(linha)
            descripcaocoluna=linha.split(',')[2].replace('P17-1','').strip()
            numero=float(linha.split(',')[1].split('(')[1].split(')')[0])
            nomecoluna=linha.split('.')[1].split(',')[0].strip().replace(' ','').replace('_','').lower()

            if 'A' in linha.split('.')[1].split(',')[1] or numero>32:
                tipocoluna='string'
            elif numero<32 and numero > 16:
                tipocoluna='bignumeric'
            else :
                tipocoluna='numeric'
            columna_raw=columna_raw+'\t'+nomecoluna+',\n'
            columna_case=columna_case+'\t'+case_column_ax.replace('#nomecoluna#',nomecoluna).replace('#unique_key#',unique_key)+'\n'
            columns_description=columns_description+'\t'+description_column_ax.replace('#nomecoluna#',nomecoluna).replace('#descripcaocoluna#',descripcaocoluna)+'\n'
            columns=columns+'\t'+estrutura_column_ax.replace('#nomecoluna#',nomecoluna).replace('#tipocoluna#',tipocoluna)+'\n'
        linha = arquivo.readline()
        count=count+1





In [ ]:
#print(columns_description)
#print(columns)
for item in arquivos_a_processar:
    arquivo = item.get('arquivo')
    arquivo_nome=arquivo
    with open(endereco_source+arquivo, "r") as arquivo:
        conteudo = arquivo.read()
        conteudo=conteudo.replace('#tabela#',tabela).replace('#table_description#',table_description).replace('#developer#',developer).replace('#unique_key#',unique_key).replace('#unique_key_lista#',unique_key_lista).replace('#columns_description#',columns_description).replace('#scheduler#',scheduler).replace('#columns#',columns).replace('#case#',columna_case).replace('#date#',data_number).replace('#path#',path).replace('#columna_raw#',columna_raw).replace('#unique_key_lista#',unique_key_lista)
        #print(conteudo)
        print(tabela+arquivo_nome)
        f = open(endereco_destino+tabela+arquivo_nome, "a")
        f.write(conteudo)
        f.close()